## Automated tutoring script induction

This notebook implements a workflow for automated question generation and tutoring script assembly.

The lesson text example was adapted from Chapter 4.3 of: Clark, M.A., Douglas, M., Choi, J.: Biology 2e. OpenStax (2018)

In [ ]:
# imports
import re
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
# load environment variables and prepare chat API
env_vars = {}
with open("../src/env.js", 'r') as file:
    for line in file:
        match = re.match(r"export const (\w+) = \"(.+?)\";", line.strip())
        if match:
            var_name, var_value = match.groups()
            env_vars[var_name] = var_value

assert env_vars["AZURE_API_KEY"] != "-", "Add your API credentials to enj.js"

# use temperature = 0.0 for deterministic model behavior
chat = AzureChatOpenAI(
    deployment_name=env_vars["AZURE_DEPLOYMENT_ID"],
    openai_api_key=env_vars["AZURE_API_KEY"],
    openai_api_base=env_vars["AZURE_ENDPOINT"],
    openai_api_version='2023-05-15', temperature=0.0)

Add reference lesson text for tutoring script induction here:

In [ ]:
SUBJECT = "biology"  # context information for prompt

LESSON_TEXT = '''“Form follows function” is a principle that many industries follow. In architecture, this principle implies that buildings should be constructed to support the activities housed within them. For example, a skyscraper should include several elevator banks. A hospital should have an easily accessible emergency room.

Our natural world also utilizes the principle of form following function, especially in cell biology. The structure of cell organelles evolved to support their specialized functions. Similar to how organs in your body function, each organelle is a specialized compartment inside the cell with a unique role to perform. For example, in the pancreas there are cells that are responsible for producing digestive enzymes, which are specific types of proteins that help with food digestion. In these pancreas cells, ribosomes - the organelles responsible for protein synthesis - are found in high numbers. This illustrates the principle of form following function: The form (high number of ribosomes) follows the function (protein production). In addition to ribosomes, cells contain many other organelles, each with its distinct function. Throughout this lesson, we'll explore several of these organelles and understand how their specific form supports their function.


Ribosomes

Ribosomes are the cell organelles responsible for protein synthesis. Inside the cell, ribosomes may group together into clusters (polyribosomes) or individual ribosomes may float freely in the cytoplasm. Ribosomes are large protein and RNA complexes, each consisting of two subunits, one large and one small (Figure 1). Ribosomes receive their “orders” for protein synthesis from the nucleus that starts the process by transcribing the DNA into messenger RNA (mRNA). After transcription, the mRNA exits the nucleus and travels to the ribosomes located in the cytoplasm. The ribosomes then translate the code provided by the sequence of the nitrogenous bases in the mRNA into a specific order of amino acids linked together to form proteins. Amino acids are the building blocks of proteins and are vital for various cellular functions (e.g., digestion).


Mitochondria

Mitochondria (singular = mitochondrion) are bean-shaped structures each with a double membrane. Mitochondria are often called “powerhouses” of cells because they make adenosine triphosphate (ATP), which is the cell’s main energy-carrying molecule. ATP represents the cell's short-term stored energy. Mitochondria conduct cellular respiration, which is the process that makes ATP using the chemical energy contained in nutrients such as glucose. The cellular respiration process takes in oxygen to break down glucose and outputs ATP plus the side products carbon dioxide (CO2) and water (H2O). In fact, the carbon dioxide that you exhale with every breath comes from the cellular reactions that produce carbon dioxide as a byproduct.

In keeping with our theme of form following function, it is important to note that muscle cells have a very high concentration of mitochondria that produce ATP. Your muscle cells need considerable energy to keep your body moving. When your cells don’t get enough oxygen, they make small amounts of ATP and also produce lactic acid.


Peroxisomes

Peroxisomes are small, round organelles enclosed by single membranes. They carry out oxidation reactions that break down fatty acids and amino acids, which are vital for various cellular functions. Oxidation reactions are also crucial in detoxifying poisons that may enter the body. Notably, many of these oxidation reactions produce hydrogen peroxide (H2O2), a substance that can be damaging to cells. However, these reactions are safely confined inside the peroxisomes by its membrane. Within the peroxisomes, enzymes break down the H2O2 into oxygen and water, mitigating potential damage. In liver cells, peroxisomes are particularly important as they detoxify alcohol. Another significant detoxification reaction carried out by peroxisomes involves the breakdown of environmental toxins like formaldehyde, commonly found in various household products and known for its harmful effects on human health.'''

### Question generation

In [ ]:
# question generation prompt
comb_rev_quest_gen_string = """You are a {subject} professor that prepares review/guiding \
questions to help your students learn a lesson. Write at least 5 free-response questions \
designed to help your students understand the lesson material. Overall, the questions you write \
should promote comprehensive learning and cover all the lesson material. Please try \
to avoid writing questions that overlap in content. The lesson material is provided \
below delimited by triple backticks.\n Write the questions in the following format:\n\
Question 1: <Question 1 text> \n\
... \n\
Question N: <Question N text> \n\n\
Lesson Material:```{content}```"""
comb_rev_quest_template = ChatPromptTemplate.from_template(comb_rev_quest_gen_string)
print(comb_rev_quest_gen_string)

In [ ]:
prompt = comb_rev_quest_template.format_messages(content=LESSON_TEXT, subject=SUBJECT)
rev_quest = chat(prompt).content
REV_QUESTS = [s.split(": ")[1] for s in rev_quest.split("\n") if len(s) > 5]
REV_QUESTS

In [ ]:
# process information for next prompt
COMB_STRING = ""
for i, q in enumerate(REV_QUESTS):
    COMB_STRING += "Question " + str(i + 1) + ": " + q + "\n"
print(COMB_STRING)

### Solution generation

In [ ]:
# solution generation prompt
rev_quest_gen_string = """You are a {subject} professor that prepares solutions \
for a range of review/guiding questions designed to help your students learn a lesson. \
The questions and lesson text are provided below delimited by triple backticks. \
The solutions should be focused and explain only the most important information from 
the lesson material. Do not just copy sentences from the lesson text. Write the 
solutions in the following format:\n\
Solution 1: <Question 1 solution text> \n\
... \n\
Solution N: <Question N solution text> \n\

Question List: ```{question}```\n\n\
Lesson Material: ```{content}```
"""
rev_answer_template = ChatPromptTemplate.from_template(rev_quest_gen_string)
print(rev_quest_gen_string)

In [ ]:
prompt = rev_answer_template.format_messages(question=COMB_STRING, content=LESSON_TEXT, subject=SUBJECT)
rev_sol = chat(prompt).content
SOLUTIONS = [s.split(": ")[1] for s in rev_sol.split("\n") if len(s) > 5]
SOLUTIONS

In [ ]:
# process information for next prompt
QS_STRING = ""
for i, q in enumerate(REV_QUESTS):
    QS_STRING += "Question " + str(i + 1) + ": " + q + "\n"
    QS_STRING += "Solution " + str(i + 1) + ": " + SOLUTIONS[i] + "\n"
print(QS_STRING)

### Expectation generation

In [ ]:
# expectation generation prompt
exp_quest_gen_string = """You are a {subject} professor that creates lists summarizing \
the key facts contained in the solutions to review/guiding questions \
designed to help your students learn a lesson. \
The questions and solutions are provided below delimited by triple backticks. \
You want to keep the lists brief and focused. \
Write the lists in the following format:\n\
List 1: <Question 1 fact 1; ...; Question 1 fact m1>\n\
...\n\
List N: <Question N fact 1; ...; Question N fact mN>\n\
\n\
Questions and Solutions: ```{questions}```
"""
exp_answer_template = ChatPromptTemplate.from_template(exp_quest_gen_string)

In [ ]:
prompt = exp_answer_template.format_messages(questions=QS_STRING, subject=SUBJECT)
rev_sol = chat(prompt).content
print(rev_sol)

### Assemble tutoring script

In [ ]:
# script assembly code
formated = [s.split(": ")[1] for s in rev_sol.split("\n") if len(s) > 5]

for i, point in enumerate(formated):
    print("Topic " + str(i + 1) + ": " + REV_QUESTS[i].replace('"', "''"))
    for j, fact in enumerate(point.split(";")):
        fact = fact.replace('"', "''")
        fact = fact[min(j, 1):]
        print("Fact " + str(i + 1) + "." + str(j + 1) + ": " + fact + ".")
    print("")

The final tutoring script is used to configure the student agent. You can add the created content as a new file in src/components/content and create a new agent definition in src/components/open_ai/